# (Quantum) Neural Networks

This notebook demonstrates the different generic (quantum) neural network implementations provided in Qiskit Machine Learning.
The networks are meant as application-anostic computational units that can be used in different use cases. 
Depending on the application, a particluar type of network might more or less suitable.
In the following, the different available networks will be discussed in more detail:

1. `NeuralNetwork`: The interface for neural networks.
2. `OpflowQNN`: A network based on the evaluation of quantum mechanical observables.
3. `TwoLayerQNN`: A special `OpflowQNN` implementation for convenience. 
3. `CircuitQNN`: A network based on the samples resulting from measuring a quantum circuit.

In [1]:
import numpy as np

from qiskit import Aer, QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit.opflow import StateFn, PauliSumOp, AerPauliExpectation, ListOp, Gradient
from qiskit.utils import QuantumInstance

In [2]:
# set method to calculcate expected values
expval = AerPauliExpectation()

# define gradient method
gradient = Gradient()

# define quantum instances (statevector and sample based)
qi_sv = QuantumInstance(Aer.get_backend('statevector_simulator'))
qi_qasm = QuantumInstance(Aer.get_backend('qasm_simulator'), shots=100) 

## 1. `NeuralNetwork`

The `NeuralNetwork` represents the interface for all neural networks available in Qiskit Machine Learning.
It just exposes a forward and a backward pass taking the data samples and trainable weights as input.
A `NeuralNetwork` does not contain any training capabilities, these are pushed to the actual algorithms / applications. Thus, a `NeuralNetwork` also does not store the values for trainable weights. In the following, different implementations of this interfaces are introduced.

Suppose a `NeuralNetwork` called `nn`.
Then, the `nn.forward(input, weights)` pass takes either flat inputs for the data and weights of size `nn.num_inputs` and `nn.num_weights`, respectively, or corresponding batches.

## 2. `OpflowQNN`

In [3]:
from qiskit_machine_learning.neural_networks import OpflowQNN

In [4]:
# construct parametrized circuit
params1 = [Parameter('input1'), Parameter('weight1')]
qc1 = QuantumCircuit(1)
qc1.h(0)
qc1.ry(params1[0], 0)
qc1.rx(params1[1], 0)
qc_sfn1 = StateFn(qc1)

# construct cost operator
H1 = StateFn(PauliSumOp.from_list([('Z', 1.0), ('X', 1.0)]))

# combine operator and circuit to objective function
op1 = ~H1 @ qc_sfn1
print(op1)

ComposedOp([
  OperatorMeasurement(1.0 * Z
  + 1.0 * X),
  CircuitStateFn(
       ┌───┐┌────────────┐┌─────────────┐
  q_0: ┤ H ├┤ RY(input1) ├┤ RX(weight1) ├
       └───┘└────────────┘└─────────────┘
  )
])


In [5]:
# construct OpflowQNN with the operator, the input parameters, the weight parameters, 
# the expected value, gradient, and quantum instance.
qnn1 = OpflowQNN(op1, [params1[0]], [params1[1]], expval, gradient, qi_sv)

In [6]:
# define (random) input and weights
input1 = np.random.rand(qnn1.num_inputs)
weights1 = np.random.rand(qnn1.num_weights)

In [7]:
# QNN forward pass
qnn1.forward(input1, weights1)

array(0.45585293)

In [8]:
# QNN backward pass
qnn1.backward(input1, weights1)

(array([-1.32212928]), array([0.09418362]))

<font color="red">INCLUDE BATCHES...</font>

Combining multiple observables in a `ListOp` also allows to create more complex QNNs

In [9]:
op2 = ListOp([op1, op1])
qnn2 = OpflowQNN(op2, [params1[0]], [params1[1]], expval, gradient, qi_sv)

In [10]:
# QNN forward pass
qnn2.forward(input1, weights1)

array([0.45585293, 0.45585293])

In [11]:
# QNN backward pass
qnn2.backward(input1, weights1)

(array([[-1.32212928],
        [-1.32212928]]),
 array([[0.09418362],
        [0.09418362]]))

## 3. `TwoLayerQNN`

The `TwoLayerQNN` is a special `OpflowQNN` on $n$ qubits that consists of first a feature map to insert data and second a variational form that is trained. The default observable is $Z^{\otimes n}$, i.e., parity.

In [12]:
from qiskit_machine_learning.neural_networks import TwoLayerQNN

In [13]:
# specify the number of qubits
num_qubits = 3

In [14]:
# specify the feature map
fm = ZZFeatureMap(num_qubits, reps=2)
print(fm.draw())

     ┌───┐┌─────────────┐                                               »
q_0: ┤ H ├┤ P(2.0*x[0]) ├──■────────────────────────────────────■────■──»
     ├───┤├─────────────┤┌─┴─┐┌──────────────────────────────┐┌─┴─┐  │  »
q_1: ┤ H ├┤ P(2.0*x[1]) ├┤ X ├┤ P(2.0*(π - x[0])*(π - x[1])) ├┤ X ├──┼──»
     ├───┤├─────────────┤└───┘└──────────────────────────────┘└───┘┌─┴─┐»
q_2: ┤ H ├┤ P(2.0*x[2]) ├──────────────────────────────────────────┤ X ├»
     └───┘└─────────────┘                                          └───┘»
«                                          ┌───┐»
«q_0: ──────────────────────────────────■──┤ H ├»
«                                       │  └───┘»
«q_1: ──────────────────────────────────┼────■──»
«     ┌──────────────────────────────┐┌─┴─┐┌─┴─┐»
«q_2: ┤ P(2.0*(π - x[0])*(π - x[2])) ├┤ X ├┤ X ├»
«     └──────────────────────────────┘└───┘└───┘»
«             ┌─────────────┐                                       »
«q_0: ────────┤ P(2.0*x[0]) ├─────────────────────────────────

In [15]:
# specify the variational form
var_form = RealAmplitudes(num_qubits, reps=1)
print(var_form.draw())

     ┌──────────┐          ┌──────────┐            
q_0: ┤ RY(θ[0]) ├──■────■──┤ RY(θ[3]) ├────────────
     ├──────────┤┌─┴─┐  │  └──────────┘┌──────────┐
q_1: ┤ RY(θ[1]) ├┤ X ├──┼───────■──────┤ RY(θ[4]) ├
     ├──────────┤└───┘┌─┴─┐   ┌─┴─┐    ├──────────┤
q_2: ┤ RY(θ[2]) ├─────┤ X ├───┤ X ├────┤ RY(θ[5]) ├
     └──────────┘     └───┘   └───┘    └──────────┘


In [16]:
# specify the observable
observable = PauliSumOp.from_list([('Z'*num_qubits, 1)])
print(observable)

1.0 * ZZZ


In [17]:
# define two layer QNN
qnn3 = TwoLayerQNN(num_qubits, fm, var_form, observable, qi_sv)

In [18]:
# define (random) input and weights
input3 = np.random.rand(qnn3.num_inputs)
weights3 = np.random.rand(qnn3.num_weights)

In [19]:
# QNN forward pass
qnn3.forward(input3, weights3)

array(-0.36051753)

In [20]:
# QNN backward pass
qnn3.backward(input3, weights3)

(array([-2.34814272,  0.91500684,  3.83553135]),
 array([-0.18182864,  0.30130437,  0.14988698,  0.34246303, -0.45361321,
         0.40539456]))

## 4. `CircuitQNN`

The `CircuitQNN` is just based on a (parametrized) `QuantumCircuit`. This can take input as well as weight parameters and produces samples from the measurement. The samples can either be interpreted as probabilities or directly as a batch of binary output. In the case of probabilities, gradients can be estimated efficiently and the `CircuitQNN` provides a backward pass as well. In case of samples, differentiation is not possible and the backward pass returns `(None, None)`.

Further, the `CircuitQNN` allows to specify different `interpret` options for the measured samples:
- 'int': interprets returns the measured bitstrings as integers
- 'str': returns the measured bitstrings
- 'tuple': returns the measured bitstrings as tuples
- a `callable`: applies a callable to the tuple of binaries and returns the result

The probabilities are then aggregated accordingly.

In case of `interpret='int'` or if `interpret` is set to a callable that returns non-negative integers, the `CircuitQNN` can be configured to return a dense instead of a sparse result.

<font color="red">output shape / dense / return types need to be further discussed</font>

<font color="red">what about `return_samples`?</font>

In [21]:
from qiskit_machine_learning.neural_networks import CircuitQNN

In [22]:
qc = RealAmplitudes(num_qubits, entanglement='linear', reps=1)
print(qc.draw())

     ┌──────────┐     ┌──────────┐            
q_0: ┤ RY(θ[0]) ├──■──┤ RY(θ[3]) ├────────────
     ├──────────┤┌─┴─┐└──────────┘┌──────────┐
q_1: ┤ RY(θ[1]) ├┤ X ├─────■──────┤ RY(θ[4]) ├
     ├──────────┤└───┘   ┌─┴─┐    ├──────────┤
q_2: ┤ RY(θ[2]) ├────────┤ X ├────┤ RY(θ[5]) ├
     └──────────┘        └───┘    └──────────┘


### 4.1 Output: sparse integer probabilities

In [23]:
# specify circuit QNN
qnn4 = CircuitQNN(qc, [], qc.parameters, interpret='int', dense=False, quantum_instance=qi_qasm)

In [24]:
# define (random) input and weights
input4 = np.random.rand(qnn4.num_inputs)
weights4 = np.random.rand(qnn4.num_weights)

In [25]:
# QNN forward pass
qnn4.forward(input4, weights4)

{5: 0.07, 1: 0.03, 4: 0.33, 6: 0.16, 0: 0.3, 7: 0.11}

In [26]:
# QNN backward pass
qnn4.backward(input4, weights4)

([],
 [{0: -0.09216068046864453,
   1: 0.04148840498308931,
   2: -0.0007757408026608872,
   3: -0.00033126830711488334,
   4: -0.08904947559066703,
   5: 0.017663416168886184,
   6: -0.14822994987144422,
   7: 0.2713952938885561},
  {0: -0.13210131218575372,
   1: 0.008453894415550809,
   2: -0.007643882070999044,
   3: -7.09695856479015e-05,
   4: -0.27857283667969407,
   5: 0.0034921663771016777,
   6: 0.3502574344002816,
   7: 0.056185505329160515},
  {0: -0.37749546931187367,
   1: -0.057689685880253645,
   2: 0.014676473756966558,
   3: 0.004076900457581105,
   4: 0.31379965071241006,
   5: 0.03755295751881814,
   6: 0.04901934484249691,
   7: 0.01605982790385442},
  {0: -0.10732547775030718,
   1: 0.10732547775030715,
   2: -0.0013801826866591108,
   3: 0.0013801826866591108,
   4: -0.11471945425238239,
   5: 0.11471945425238245,
   6: -0.13154485543911065,
   7: 0.13154485543911065},
  {0: -0.04030564293046261,
   1: -0.003675137163397957,
   2: 0.04030564293046261,
   3: 0.003

### 4.2 Output: sparse string probabilities

In [27]:
# specify circuit QNN
qnn5 = CircuitQNN(qc, [], qc.parameters, interpret='str', dense=False, quantum_instance=qi_qasm)

In [28]:
# define (random) input and weights
input5 = np.random.rand(qnn5.num_inputs)
weights5 = np.random.rand(qnn5.num_weights)

In [29]:
# QNN forward pass
qnn5.forward(input5, weights5)

{'001': 0.05, '010': 0.02, '000': 0.85, '100': 0.08}

In [30]:
# QNN backward pass
qnn5.backward(input5, weights5)

([],
 [{'000': -0.13823102322492514,
   '001': -0.001105836231703604,
   '010': -0.00750328039967098,
   '011': 0.013687533359384729,
   '100': -0.018297362172265814,
   '101': -0.004402942151135472,
   '110': -0.012252553839265453,
   '111': 0.16810546465958168},
  {'000': -0.08876182466194363,
   '001': 0.019292821125997405,
   '010': 0.027721070281507967,
   '011': 0.0031423604446615913,
   '100': -0.06055188004706111,
   '101': 0.0016414145572968795,
   '110': 0.07233588822830381,
   '111': 0.02518015007123729},
  {'000': -0.3385533112232363,
   '001': -0.011334563660513282,
   '010': -0.0037145230639364767,
   '011': 0.009814061241480052,
   '100': 0.33315563586236113,
   '101': 0.005319647606825769,
   '110': 0.009112198424811502,
   '111': -0.003799145187792543},
  {'000': -0.11894402235700349,
   '001': 0.11894402235700352,
   '010': -0.0078114160063236905,
   '011': 0.007811416006323691,
   '100': -0.013712059934886934,
   '101': 0.01371205993488694,
   '110': -0.0138829580083

### 4.3 Output: dense parity probabilities

In [31]:
# specify circuit QNN
parity = lambda x: np.sum(x) % 2
output_shape = 2  # this is required in case of a callable with dense output
qnn6 = CircuitQNN(qc, [], qc.parameters, interpret=parity, dense=True, output_shape=output_shape,
                  quantum_instance=qi_qasm)

In [32]:
# define (random) input and weights
input6 = np.random.rand(qnn6.num_inputs)
weights6 = np.random.rand(qnn6.num_weights)

In [33]:
# QNN forward pass
qnn6.forward(input6, weights6)

array([[0.67, 0.33]])

In [34]:
# QNN backward pass
qnn6.backward(input6, weights6)

(array([], shape=(1, 2, 0), dtype=float64),
 array([[[-0.16637214,  0.03831956, -0.10334517, -0.19719425,
          -0.2484858 ,  0.12584155],
         [ 0.16637214, -0.03831956,  0.10334517,  0.19719425,
           0.2484858 , -0.12584155]]]))

### 4.4 Output: dense samples

In [35]:
# TODO: show how to setup a circuit QNN that returns samples... (but no gradients)

In [36]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright